# Separate from command line

In [5]:
from IPython.display import Audio
import os

from spleeter.separator import Separator

path = "singer_voices\singer_music"
for f in os.listdir(path):
  if os.path.isfile(os.path.join(path,f)):
    separator = Separator("spleeter:2stems")
    prediction = separator.separate_to_file(os.path.join(path,f), "singer_voices\singer")

# 'output' will contain the separated audio tracks, including vocals and accompaniment

Exception ignored in: <generator object Estimator.predict at 0x7867d7f534c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorflow_estimator/python/estimator/estimator.py", line 618, in predict
    with tf.Graph().as_default() as g:
  File "/usr/lib/python3.10/contextlib.py", line 153, in __exit__
    self.gen.throw(typ, value, traceback)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py", line 5821, in get_controller
    with super(_DefaultGraphStack,
  File "/usr/lib/python3.10/contextlib.py", line 153, in __exit__
    self.gen.throw(typ, value, traceback)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/ops.py", line 5633, in get_controller
    raise AssertionError(
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.framework.ops.Graph'> objects


INFO:spleeter:File ./io/output/audio_example/vocals.wav written succesfully


INFO:spleeter:File ./io/output/audio_example/vocals.wav written succesfully


INFO:spleeter:File ./io/output/audio_example/accompaniment.wav written succesfully


INFO:spleeter:File ./io/output/audio_example/accompaniment.wav written succesfully


## stick

In [1]:
import pydub
import os


def merge_voices():
    path = "singer_voices/singer"
    two_files = []
    for file in os.listdir(path):
        name = file.split(".")[0]
        if len(name.split("_")) == 2:
            if name.split("_")[0] not in two_files:
                two_files.append(name.split("_")[0])
                result= 0
                for f in os.listdir(path):
                    if f.split(".")[0].split("_")[0] == name.split("_")[0] :
                        result += pydub.AudioSegment.from_file(f'singer_voices/singer/{f}')
                result.export(f"singer_voices/stick_data/{name.split('_')[0]}.wav")
        else:
            result = pydub.AudioSegment.from_file(f'singer_voices/singer/{file}')
            result.export(f"singer_voices/stick_data/{name}.wav", format="wav")

if __name__ == "__main__":
    merge_voices()

## Remove silence

In [5]:
import pydub
import os

def remove_silence_and_to_wav():
    files = os.listdir('singer_voices/stick_data')
    for file in files:
        audio = pydub.AudioSegment.from_file(os.path.join('singer_voices/stick_data', file))

        chunks = pydub.silence.split_on_silence(audio, min_silence_len=500, silence_thresh=-25)
        result = sum(chunks)
        file_name = file.split('.')[0]
        result.export(f'singer_voices/rm_silence/{file_name}_rm.wav', format="wav")

if __name__ == "__main__":
    remove_silence_and_to_wav()

## Split to 1sec data

In [6]:
import pydub
import os


def split_voices():
    for file in os.listdir('singer_voices/rm_silence'):
        audio = pydub.AudioSegment.from_file(os.path.join('singer_voices/rm_silence', file))
        chunks = pydub.utils.make_chunks(audio, 1000)
        person_name = file.split('.')[0]
        os.makedirs(os.path.join('singer_voices/dataset',person_name), exist_ok=True)

        for i,chunk in enumerate(chunks):
            if len(chunk) >= 1000:
                chunk.export(os.path.join('singer_voices/dataset',person_name, f'voice_{i}.wav'), format="wav")

if __name__ == "__main__":
    split_voices()